If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
#from huggingface_hub import notebook_login

#notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.28.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
#from transformers.utils import send_example_telemetry

#send_example_telemetry("token_classification_notebook", framework="pytorch")

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](images/token_classification.png)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es" #"PlanTL-GOB-ES/bsc-bio-ehr-es"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [ ]:
#datasets = load_dataset("conll2003")
#datasets = load_dataset("/content/drive/MyDrive/CLEF2023/custom_dataset")
from datasets import DatasetDict

dataset_split = load_dataset('/content/drive/MyDrive/CLEF2023/custom_dataset', split=[
    'train[0%:90%]', 'train[90%:100%]'
])

datasets = DatasetDict({'train': dataset_split[0], 'validation': dataset_split[1]})

Generating train split: 0 examples [00:00, ? examples/s]

Dataset custom_dataset downloaded and prepared to /root/.cache/huggingface/datasets/custom_dataset/customconll2003/1.0.0/5c1a1699f8c949ed42bb8da4620f3179206b04e5d13ccc3d5b20ebb72744bd52. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 674
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 75
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
datasets["train"][0]

{'id': '0',
 'tokens': ['Paciente',
  'de',
  '70',
  'años',
  'de',
  'edad,',
  'minero',
  'jubilado,',
  'sin',
  'alergias',
  'medicamentosas',
  'conocidas,',
  'que',
  'presenta',
  'como',
  'antecedentes',
  'personales:',
  'accidente',
  'laboral',
  'antiguo',
  'con',
  'fracturas',
  'vertebrales',
  'y',
  'costales;',
  'intervenido',
  'de',
  'enfermedad',
  'de',
  'Dupuytren',
  'en',
  'mano',
  'derecha',
  'y',
  'by-pass',
  'iliofemoral',
  'izquierdo;',
  'Diabetes',
  'Mellitus',
  'tipo',
  'II,',
  'hipercolesterolemia',
  'e',
  'hiperuricemia;',
  'enolismo',
  'activo,',
  'fumador',
  'de',
  '20',
  'cigarrillos',
  '/',
  'día.',
  'Es',
  'derivado',
  'desde',
  'Atención',
  'Primaria',
  'por',
  'presentar',
  'hematuria',
  'macroscópica',
  'postmiccional',
  'en',
  'una',
  'ocasión',
  'y',
  'microhematuria',
  'persistente',
  'posteriormente,',
  'con',
  'micciones',
  'normales.',
  'En',
  'la',
  'exploración',
  'física',
  'prese

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [ ]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PROCEDIMIENTO', 'I-PROCEDIMIENTO'], id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PROCEDIMIENTO', 'I-PROCEDIMIENTO']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,583,"[Varón, de, 45, años, de, edad,, sin, alergias, medicamentosas, conocidas., Entre, sus, antecedentes, personales:, exfumador, (más, de, 30, paquetes/año),, diabetes, mellitus, tipo, 2,, hipercolesterolemia,, una, historia, de, anticuerpo, antifosfolípido, circulante, con, antecedentes, de, dos, trombosis, venosas, profundas., Dentro, de, su, historia, vascular, previa, al, ingreso:, en, octubre, del, 2004, se, le, había, colocado, un, stent, ilíaco, izquierdo, más, un, by-pass, fémoro-femoral, izquierda-derecha., En, febrero, del, 2005, se, le, realizó, dilatación, con, crioterapia, de, ilíaca, izquierda,, que, no, es, efectiva, por, lo, que, se, le, realiza, by-pass, fémoro-poplíteo, a, primera, porción, en, el, miembro, inferior, izquierdo., En, febrero, del, 2006, ingresa, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...]"
1,312,"[Mujer, de, 45, años, que, es, remitida, desde, el, servicio, de, neurocirugía, por, pérdida, de, visión, súbita, bilateral, tras, cirugía, de, derivación, ventrículoperitoneal, de, un, higroma, subdural, temporal, y, absceso, occipital, craneal., Todo, ello, resultado, de, una, complicación, postquirúrgica, subaguda., Quince, días, antes, había, sido, intervenida, de, descompresión, preventiva, medular, por, Malformación, de, Arnold-Chiari, I., La, paciente, contaba, con, una, clínica, de, 16, años, de, evolución,, con, cefaleas, de, intensidad, leve,, inestabilidad,, mareos, rotatorios,, vómitos, autolimitados, de, duración, variable, y, nistagmus, vertical, leve, desde, hacía, un, año., Éste, último, ayudó, a, la, sospecha, y, diagnóstico, de, la, malformación, congénita,, ya, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...]"
2,235,"[Varón, de, 51, años, con, enfermedad, de, Crohn, ileocólica, de, larga, evolución,, con, múltiples, complicaciones, por, brotes, y, que, precisó, cirugía, de, resección, intestinal, quedando, con, una, ileostomía, terminal, en, 2009., Este, año, se, inicia, NPD, mediante, un, dispositivo, intravascular, totalmente, implantado., Se, encontraba, en, tratamiento, con, Infliximab, y, corticoides., El, año, siguiente, ingresa, en, varias, ocasiones, por, síndrome, febril, asociado, a, dolor, abdominal,, sin, un, foco, evidente, y, considerándose, el, origen, la, propia, enfermedad., En, tales, hospitalizaciones, se, aisló, Staphylococcus, epidermidis, en, varios, hemocultivos., También, se, sustituyó, el, catéter, por, rotura,, retirándose, solamente, el, reservorio, y, la, porción, proximal, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, B-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, ...]"
3,642,"[Presentamos, el, caso, de, una, niña, de, 4, años, que, consultó, por, 

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [0, 428, 4354, 15, 8237, 1658, 14719, 1308, 3507, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [0, 428, 4354, 50288, 8237, 1658, 14719, 1308, 3507, 274, 586, 401, 12250, 14478, 30953, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['Varón', 'de', '36', 'años,', 'sin', 'antecedentes', 'de', 'interés,', 'que', 'fue', 'estudiado', 'en', 'la', 'consulta', 'de', 'medicina', 'interna', 'por', 'presentar', 'masa', 'inguinoescrotal', 'izquierda', 'dolorosa', 'a', 'la', 'palpación', 'de', 'dos', 'meses', 'de', 'evolución,', 'sin', 'pérdida', 'de', 'peso', 'ni', 'síndrome', 'miccional.', 'A', 'la', 'exploración,', 'los', 'testes', 'eran', 'de', 'tamaño', 'y', 'consistencia', 'normales,', 'con', 'un', 'cordón', 'espermático', 'izquierdo', 'indurado', 'y', 'muy', 'doloroso.', 'La', 'ecografía', 'testicular', 'fue', 'normal.', 'La', 'CT', 'de', 'abdomen-pelvis', 'reveló', 'masa', 'de', '6', 'x', '3', 'centímetros', 'en', 'el', 'trayecto', 'del', 'cordón', 'espermático', 'izquierdo', 'sin', 'objetivarse', 'imágenes', 'de', 'afectación', 'retroperitoneal.', 'Con', 'el', 'diagnóstico', 'de', 'tumor', 'paratesticular', 'izquierdo', 'fue', 'intervenido,', 'encontrándose', 'una', 'masa', 'en', 'cordón', 'espermático', 'y', 'realiz

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', 'ĠVarÃ³n', 'Ġde', 'Ġ36', 'ĠaÃ±os', ',', 'Ġsin', 'Ġantecedentes', 'Ġde', 'ĠinterÃ©s', ',', 'Ġque', 'Ġfue', 'Ġestudiado', 'Ġen', 'Ġla', 'Ġconsulta', 'Ġde', 'Ġmedicina', 'Ġinterna', 'Ġpor', 'Ġpresentar', 'Ġmasa', 'Ġingu', 'ino', 'esc', 'ro', 'tal', 'Ġizquierda', 'Ġdolorosa', 'Ġa', 'Ġla', 'ĠpalpaciÃ³n', 'Ġde', 'Ġdos', 'Ġmeses', 'Ġde', 'ĠevoluciÃ³n', ',', 'Ġsin', 'ĠpÃ©rdida', 'Ġde', 'Ġpeso', 'Ġni', 'ĠsÃŃndrome', 'Ġmiccional', '.', 'ĠA', 'Ġla', 'ĠexploraciÃ³n', ',', 'Ġlos', 'Ġtes', 'tes', 'Ġeran', 'Ġde', 'ĠtamaÃ±o', 'Ġy', 'Ġconsistencia', 'Ġnormales', ',', 'Ġcon', 'Ġun', 'ĠcordÃ³n', 'Ġesper', 'mÃ¡tico', 'Ġizquierdo', 'Ġind', 'urado', 'Ġy', 'Ġmuy', 'Ġdoloroso', '.', 'ĠLa', 'ĠecografÃŃa', 'Ġtesticular', 'Ġfue', 'Ġnormal', '.', 'ĠLa', 'ĠCT', 'Ġde', 'Ġabdomen', '-', 'pel', 'vis', 'ĠrevelÃ³', 'Ġmasa', 'Ġde', 'Ġ6', 'Ġx', 'Ġ3', 'ĠcentÃŃmetros', 'Ġen', 'Ġel', 'Ġtrayecto', 'Ġdel', 'ĠcordÃ³n', 'Ġesper', 'mÃ¡tico', 'Ġizquierdo', 'Ġsin', 'Ġobjetivarse', 'ĠimÃ¡genes', 'Ġde', 'ĠafectaciÃ³n', 'Ġret

Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(179, 227)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 20, 20, 20, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 37, 38, 39, 40, 40, 41, 42, 42, 43, 44, 45, 46, 47, 48, 48, 49, 50, 51, 52, 52, 53, 54, 54, 55, 56, 57, 57, 58, 59, 60, 61, 62, 62, 63, 64, 65, 66, 66, 66, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 79, 80, 81, 82, 83, 84, 85, 86, 86, 87, 88, 89, 90, 91, 92, 92, 92, 93, 94, 95, 95, 96, 97, 98, 99, 100, 101, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 112, 112, 112, 113, 114, 114, 114, 115, 116, 117, 118, 119, 120, 120, 120, 121, 122, 123, 123, 124, 125, 126, 127, 128, 129, 129, 130, 130, 130, 131, 132, 133, 133, 133, 134, 135, 136, 136, 137, 138, 139, 140, 141, 142, 143, 144, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 155, 155, 156, 157, 157, 158, 159, 159, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 173, 174, 175, 176, 177, 178, 178, Non

As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

227 227


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [ ]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[0, 3106, 262, 3210, 610, 262, 1172, 15, 1363, 687, 48257, 15, 526, 9321, 20472, 8463, 15, 300, 1752, 416, 3214, 5135, 29, 6208, 3560, 10430, 299, 7517, 17240, 290, 30318, 30, 15035, 262, 834, 262, 49493, 817, 11657, 288, 3428, 2550, 290, 4585, 16, 16624, 45765, 47039, 2729, 30, 6612, 24567, 981, 2974, 15, 17546, 277, 37330, 30, 288, 10132, 4540, 15, 14334, 262, 634, 10874, 759, 1010, 17, 438, 9074, 794, 3044, 3988, 344, 2966, 14005, 36994, 2857, 4324, 1240, 288, 363, 7234, 290, 2188, 26858, 10490, 9128, 4647, 15, 299, 44912, 3719, 17, 452, 284, 3904, 2469, 1752, 308, 1087, 1302, 1217, 15, 299, 5751, 290, 13399, 3719, 30, 14168, 11507, 6831, 299, 21426, 262, 5967, 2196, 414, 18, 3624, 17, 452, 284, 4464, 262, 3604, 4632, 284, 4320, 262, 588, 19511, 18, 2772, 290, 762, 16, 24, 7254, 18, 18382, 30, 2693, 262, 22519, 1599, 17, 21220, 1599, 30, 288, 284, 15850, 4632, 363, 10508, 262, 29781, 828, 18, 3877, 290, 12701, 262, 3789, 25, 828, 18, 3877, 30, 1719, 5351, 290, 2034, 1

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/674 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['classifier

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir = f"/content/drive/MyDrive/CLEF2023/{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy = "epoch",
    #load_best_model_at_end=True,
    #metric_for_best_model="accuracy",
    push_to_hub=False,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
metric = load_metric("seqeval")

<ipython-input-32-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


This metric takes list of labels for the predictions and references:

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'PROCEDIMIENTO': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 9},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.183960,0.347973,0.177281,0.234892,0.945956
2,No log,0.137498,0.424444,0.328744,0.370514,0.952114
3,No log,0.131894,0.361795,0.430293,0.393082,0.951839


TrainOutput(global_step=129, training_loss=0.22315674419550932, metrics={'train_runtime': 224.9399, 'train_samples_per_second': 8.989, 'train_steps_per_second': 0.573, 'total_flos': 528345798128604.0, 'train_loss': 0.22315674419550932, 'epoch': 3.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.13189402222633362,
 'eval_precision': 0.361794500723589,
 'eval_recall': 0.43029259896729777,
 'eval_f1': 0.39308176100628933,
 'eval_accuracy': 0.9518387285424336,
 'eval_runtime': 3.0689,
 'eval_samples_per_second': 24.439,
 'eval_steps_per_second': 1.629,
 'epoch': 3.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'PROCEDIMIENTO': {'precision': 0.361794500723589,
  'recall': 0.43029259896729777,
  'f1': 0.39308176100628933,
  'number': 581},
 'overall_precision': 0.361794500723589,
 'overall_recall': 0.43029259896729777,
 'overall_f1': 0.39308176100628933,
 'overall_accuracy': 0.9518387285424336}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
#trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("sgugger/my-awesome-model")
```